In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
COLAB = True
print("Note: using Google CoLab")

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install wandb

In [ ]:
!pip install albumentations==0.4.6

In [ ]:
!python -m pip install -U scikit-image

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import wandb
wandb.login()

# Train Model


In [ ]:
import os

MODEL_TYPE = "EfficientNet"
RUN_NAME = "RUN_1"
PROJECT_NAME = "CoVision-Classification"

CHECKPOINT_DIR_BASE = "/content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/experiments"
CHECKPOINT_DIR = os.path.join(CHECKPOINT_DIR_BASE, PROJECT_NAME, RUN_NAME)

TRAIN_DATA_PATH = "/content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/data/train"
TEST_DATA_PATH = "/content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/data/test"
LOAD_SIZE = 256
MAX_EPOCHS = 150

TRAIN_PATH = "/content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/lightning_ai/trainer.py"

cmd = f"/usr/bin/python3 {TRAIN_PATH} --checkpoint_dir {CHECKPOINT_DIR} --run_name {RUN_NAME} --project_name {PROJECT_NAME} --train_data_path {TRAIN_DATA_PATH} --test_data_path {TEST_DATA_PATH} --load_size {LOAD_SIZE} --max_epochs {MAX_EPOCHS}"
!{cmd}

# Load best model in pytorch lightning and export to onnx format

## Create ClassificationNet class

In [34]:
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F
import pytorch_lightning as pl
from efficientnet_pytorch import EfficientNet

from sklearn import metrics


class ClassificationNet(pl.LightningModule):

    def __init__(
        self,
        num_classes: int=4,
        lr: float = 1e-3,
):
        super().__init__()
        self.num_classes = num_classes
        self.lr = lr
        self.net = EfficientNet.from_pretrained("efficientnet-b2", in_channels = 3, num_classes = num_classes)
        self.loss_function = nn.CrossEntropyLoss()

        # save hyper-parameters to self.hparams (auto-logged by W&B)
        self.save_hyperparameters()

    def forward(self, x):

        return self.net(x)

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.net.parameters(), lr=self.lr)
        return opt

    def training_step(self, batch, batch_nb):

        img, target = batch

        output = self(img)

        loss = self.loss_function(output, target)

        self.log('Train - CrossEntropyLoss', loss, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_nb):

        img, target = batch

        output = self(img)

        loss = self.loss_function(output, target)

        #######
        # Calculate different metrics here: (F1 Score, Precision, Recall, Sensitivity, Specifity etc.)

        # Shape of output: (B, C) with C= number of classes and B = batch size so e.g. (16,4)
        # , each value is a "probability" (after using nn.Softmax()) for the specific class e.g. output[0] = [7.1723e-01, 1.8264e-03, 2.7491e-01, 6.0351e-03]
        # Shape of target: (B) the value tells which class it is
        
        prediction = torch.argmax(output, dim=1)
        
        prediction = prediction.clone().cpu()
        target = target.clone().cpu()

        accuracy = metrics.accuracy_score(target, prediction)
        conf_mat = metrics.confusion_matrix(target, prediction)

        precision_micro = metrics.precision_score(target, prediction, average='micro')
        precision_macro = metrics.precision_score(target, prediction, average='macro')
        recall_micro = metrics.recall_score(target, prediction, average='micro')
        recall_macro = metrics.recall_score(target, prediction, average='macro')


        ############

        self.log('Val - CrossEntropyLoss', loss, prog_bar=True)
        self.log('Accuracy', accuracy, prog_bar=True)
        self.log('Precision micro', precision_micro, prog_bar=True)
        self.log('Precision macro', precision_macro, prog_bar=True)
        self.log('Recall micro', recall_micro, prog_bar=True)
        self.log('Recall macro', recall_macro, prog_bar=True)
        # self.log('Conf_mat', conf_mat, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):

        img, target = batch

        output = self(img)

        loss = self.loss_function(output, target)

        #######
        # Calculate different metrics here: (F1 Score, Precision, Recall, Sensitivity, Specifity etc.)

        print()
        print("shape of output: ", output.shape())
        print()
        print("shape of target: ", target.shape())
        print()


        ############

        self.log('Test - CrossEntropyLoss', loss, prog_bar=True)

        return loss
    


## Load model

In [68]:
%cd MyDrive/

/content/drive/MyDrive


In [69]:
%cd CoVision/Github/CoVision/classification_model_training/experiments/CoVision-Classification/RUN_1/

/content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/experiments/CoVision-Classification/RUN_1


### Export the efficientNet model to ONNX

In [70]:
import torch
path_to_model = r"/content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/experiments/CoVision-Classification/RUN_1/epoch=173-step=3132.ckpt"
model = ClassificationNet.load_from_checkpoint(path_to_model)

model.net.set_swish(memory_efficient=False)
input_sample = torch.randn((16, 3, 256, 256))

# torch.onnx.export(model,                     # model being run
#                   ##since model is in the cuda mode, input also need to be
#                   input_sample,              # model input (or a tuple for multiple inputs)
#                   "model_troch_export.onnx", # where to save the model (can be a file or file-like object)
#                   export_params=True,        # store the trained parameter weights inside the model file
#                   opset_version=10,          # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
#                   input_names = ['input'],   # the model's input names
#                   output_names = ['output'], # the model's output names
#                   dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})


filepath = "model.onnx"
input_sample = torch.randn((16, 3, 256, 256))
model.to_onnx(filepath, input_sample, export_params=True)


Loaded pretrained weights for efficientnet-b2


# convert .onnx to tensorflow.js format

In [ ]:
!pip install onnx-tf

In [ ]:
PATH_EXP = r"/content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/experiments/CoVision-Classification/RUN_1/"

INPUT_MODEL = os.path.join(PATH_EXP, "model.onnx")
OUTPUT_MODEL = os.path.join(PATH_EXP, "model.pb")


import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load(INPUT_MODEL)  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph(OUTPUT_MODEL)  # export the model

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names=tfjs_layers_model \
    --saved_model_tags=serve \
    /content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/experiments/CoVision-Classification/RUN_1/model.pb \
    /content/drive/MyDrive/CoVision/Github/CoVision/classification_model_training/experiments/CoVision-Classification/RUN_1/